In [1]:
!sudo apt-get install libmagickwand-dev
!pip install --no-cache-dir \
    opencv-python-headless==4.6.*\
    rawpy==0.17.* \
    pandas \
    Pillow==7.1.2 \
    scikit-image==0.16.2 \
    scipy \
    tqdm \
    Wand

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmagickwand-dev is already the newest version (8:6.9.7.4+dfsg-16ubuntu6.14).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!git clone https://github.com/andreacos/BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation

fatal: destination path 'BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation' already exists and is not an empty directory.


In [3]:
import io
import os
import sys
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd
import warnings

from glob import glob
from PIL import Image
from skimage.util import view_as_windows

warnings.filterwarnings("ignore")

In [4]:
# GET PATH
sys.path.insert(1, "BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation")

In [5]:
from networks import custom_two_terms_loss_wrapper, custom_softmax_activation, custom_mse_wrapper
from utils import max_min_coefficient, label2coefficient, string2Q

In [6]:
model_path = "BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/models/"
data_path = "BoostingCNN-Jpeg-Primary-Quantization-Matrix-Estimation/resources/"
img_path = "test_images/"
model_file = model_path + 'model_QF1_60-98_QF2_90-2-term-loss.h5'
img_file = data_path + '00000000_redaf7d93t.TIF_85_90.png'

# Загрузим таблицу, связывающую каждую пару факторов качества JPEG с соответствующими коэффициентами Q
qf_map = np.load(data_path + 'qf1_qf2_map_90.npy', allow_pickle=True)

Набор пар коэффициентов качества JPEG (QF1, QF2). Коэффициент качества первого сжатия JPEG варьируется в заданном диапазоне (например, QF1 = [60, 70, 80, 90]), в то время как коэффициент качества второго сжатия JPEG является фиксированным (например, QF2 = 90).

In [7]:
examples = [
    (60,90),
    (70,90),
    (80,90),
    (90,90),
    (98,90)
]

In [8]:
# Предобработка изображений
def preprocess_input(im, target_size, scale=255.):
    if type(im) == str :
      im = cv2.imread(im)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2YCrCb)[:,:,0]
    if im.shape != target_size:
        im = cv2.resize(im, target_size)
    return im.astype(np.float32) / scale

In [9]:
# Расчёт MSE
def mse (predictions, k_dct):
  result = np.mean((k_dct - predictions) **2, axis=1)
  return np.argmin(result)

In [10]:
# Расчёт weighted MSE (Пункт А3)
def mse_weighted(predictions, k_dct, weights):
  diff = np.asarray((predictions - k_dct)**2)
  mult = diff*weights
  weighted = np.average(mult, axis=1, weights=weights)
  return np.argmin(weighted)

In [11]:
# Сжатие изображения
def compress_image(img, *qfs):
  for qf in qfs:
    jpeg_encoded = cv2.imencode('.jpg', img, [int(cv2.IMWRITE_JPEG_QUALITY), qf])[1]
    jpeg_encoded_image = Image.open(io.BytesIO(jpeg_encoded))
    img = np.array(jpeg_encoded_image)
  return img

In [12]:
# Тестовый кейс
def test_case():
  x = preprocess_input(img_file, (64, 64), 255.)
  prediction = model.predict(np.expand_dims(x, [0, -1]))
  # label2coefficient() используется для декодирования каждого прогнозирования с горячим 
  # кодированием на основе максимального значения, которое может быть присвоено каждому коэффициенту JPEG
  predicted_label = label2coefficient(prediction.flatten(), max_coefficients=max_coeffs)
  print(predicted_label)

In [13]:
# Веса для Weighted MSE
q50_coeffs = np.array([16,11,12,14,12,10,16,14,13,14,18,17,16,19,24], dtype='int64')

In [14]:
# Прогнозирование
def predict(img, k_dct, max_coeffs):
  prediction = model.predict(np.expand_dims(img, [0, -1]), verbose=0)
  predicted_label = label2coefficient(prediction.flatten(), max_coefficients=max_coeffs)
  t_mse = mse(predicted_label,k_dct)
  w_mse = mse_weighted(predicted_label, k_dct, q50_coeffs)
  result = labels[t_mse]
  weighted = labels[w_mse]
  return result, weighted

In [15]:
# Максимальное значение для коэффициентов
max_coeffs, _ = max_min_coefficient(quality_range=(50, 100),
                                    n_coeffs=15,
                                    zig_zag_order=True)
model = tf.keras.models.load_model(model_file,
                                    custom_objects=({'custom_softmax': custom_softmax_activation(max_coeffs),
                                                    'custom_two_terms_loss_wrapper': custom_two_terms_loss_wrapper(max_coeffs, 0.8),
                                                    'custom_mse': custom_mse_wrapper(max_coeffs)}))

In [16]:
test_case()

1/1 [==============================] - 1s 1s/step
[ 7  5  4  6  4  3  7  6  4  6  8  7  7  8 11]


In [17]:
# Готовим данные из исходных
labels = qf_map[:,:2]
k_dct = []
for rec in qf_map:
  k_dct.append(string2Q(rec[2], size=(8, 8), flatten=True)[:15])
k_dct = np.array(k_dct)

In [18]:
# A0: Программно сгенерировать 5 принципиально различных ситуаций: однократное сжатие, 𝑄𝐹1 < 𝑄𝐹2, 𝑄𝐹1 ≪ 𝑄𝐹2, 𝑄𝐹1 > 𝑄𝐹2, 𝑄𝐹1 ≈ 𝑄𝐹2. 
anls_data = []

for e in examples:
  for path in glob(img_path+'*.tif'):
    tmp = compress_image(cv2.imread(path), *e)
    tmp = tmp[:64,:64]
    tmp = preprocess_input(tmp, (64, 64), 255.)
    pred, pred_w = predict(tmp, k_dct, max_coeffs)
    anls_data.append([e[0], pred[0], np.abs(pred[0]-e[0]), pred_w[0], np.abs(pred_w[0]-e[0])])

In [19]:
df_a0 = pd.DataFrame(anls_data, columns=['QF1', 'Prediction', 'Error', 'Prediction_WMSE', 'Error_WMSE'])
df_a0_g = df_a0.groupby('QF1').mean()

In [20]:
# Вывод результатов А0+A3
df_a0_g

,Prediction,Error,Prediction_WMSE,Error_WMSE
QF1,,,,
60,62.500000,2.500000,62.583333,2.583333
70,67.750000,3.083333,67.750000,3.083333
80,78.833333,1.833333,78.833333,1.833333
90,95.000000,5.000000,95.083333,5.083333
98,95.583333,2.416667,95.666667,2.333333
